In [56]:
import parselmouth
import glob
import os
import statistics
import numpy as np
import pandas as pd
from typing import Dict, Iterator, Iterable
from parselmouth.praat import call

In [3]:
# Returned sorted file paths
def getFilePaths(path):
    files = glob.glob(path)
    files.sort(key = sortKey)
    return files

def sortKey(path):
    parts = path.split('_')
    utterance = int(parts[7][1:])
    return utterance

# Get f0min and f0max
f0min_male = 80
f0max_male = 250
f0min_female = 140
f0max_female = 310

# Get formants (from PraatScripts by https://github.com/drfeinberg/PraatScripts/, modified to remove median)
def measureFormants(file, f0min, f0max):
    sound = parselmouth.Sound(file) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    try:
        f1_mean = statistics.mean(f1_list)
        f2_mean = statistics.mean(f2_list)
        f3_mean = statistics.mean(f3_list)
        f4_mean = statistics.mean(f4_list)
    except statistics.StatisticsError:
        print("Skipping: " + file + ", no formants extracted")
        return None
    
    return f1_mean, f2_mean, f3_mean, f4_mean

# (F1_actual + F1_feedback)/F1_actual
def getPerturbationValues(actual, feedback, f0min, f0max):
    perturbation_values: Dict[str, float] = {
        "f1": [],
        "f2": [],
        "f3": [],
        "f4": [],
    }
    
    first_trial = True
    
    for i in range(1, len(actual)):
        print("Fetching formants from " + actual[i])
        
        # Get formants
        actual_formants = measureFormants(actual[i], f0min, f0max)
        feedback_formants = measureFormants(feedback[i], f0min, f0max)
        
        if actual_formants is None or feedback_formants is None:
            continue
        else:
            (f1_actual, f2_actual, f3_actual, f4_actual) = actual_formants
            (f1_feedback, f2_feedback, f3_feedback, f4_feedback) = feedback_formants
            f1_perturb = (f1_actual + f1_feedback)/f1_actual
            f2_perturb = (f2_actual + f2_feedback)/f2_actual
            f3_perturb = (f3_actual + f3_feedback)/f3_actual
            f4_perturb = (f4_actual + f4_feedback)/f4_actual
        
        perturbation_values["f1"].append(f1_perturb)
        perturbation_values["f2"].append(f2_perturb)
        perturbation_values["f3"].append(f3_perturb)
        perturbation_values["f4"].append(f4_perturb)
        
    perturbation_values = pd.DataFrame(perturbation_values)
    return perturbation_values

In [95]:
def measurePitch(file):
    sound = parselmouth.Sound(file)
    pitch = sound.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    return pitch_values

# def grump(pitch_values):
#     syllable = []
#     syllable_pitches = []
#     last_pitch = 0
#     for pitch in pitch_values:
#         # if the last pitch was not 0 but the current pitch is 0
#         if last_pitch != 0 and pitch == 0:
#             syllable_pitches.append(statistics.mean(syllable))
#             syllable = []
#             last_pitch = pitch

#         # if neither the last pitch nor the current pitch is 0
#         # and if the last pitch was 0 but the current pitch is not 0
#         elif pitch != 0:
#             syllable.append(pitch)
#             last_pitch = pitch
            
#     return syllable_pitches

def getPitchOfEachSyllable(pitches: Iterable[float]) -> Iterator[float]:                                                     
    acc = 0.0                                                                                                                 
    count = 0                                                                                                                 
    for pitch in pitches:                                                                                                     
        if pitch:                                                                                                             
            # We're in the start/middle of a syllable                                                                         
            acc += pitch                                                                                                      
            count += 1                                                                                                        
        elif count:                                                                                                           
            # We're at the end of a syllable                                                                                  
            yield acc / count                                                                                                 
            acc = 0                                                                                                           
            count = 0                                                                                                         
        else:                                                                                                                 
            # We're between syllables                                                                                         
            pass    
    if count:
        yield acc / count
        
os.chdir('/Users/letitiaho/src/talker_change_data_processing')
stim = glob.glob('0_set_up_and_raw_data/data/stim/low_pass_400/*.wav')
stim_pitches = {}
for file in stim:
    # Get pitches across utterance
    pitch_values = measurePitch(file)
    
    # Separate pitches for each syllable
    syllable_pitches = list(getPitchOfEachSyllable(pitch_values))
    
    # Save in dict
    stim_pitches[file.split('/')[-1]] = syllable_pitches

{'frog_f.wav': [176.81601943930374],
 'word_crow.wav': [97.2233215356513],
 'word_zipper.wav': [104.37567197444336, 83.25418533066467, 76.02143110714105],
 'word_dog.wav': [91.1859331524194],
 'word_cashregister.wav': [117.0743952626585,
  85.83597384438858,
  85.78389110856844],
 'zipper_f.wav': [177.36985478950297, 151.44289135919206],
 'saxophone_f.wav': [182.71456772322185,
  161.86985007909485,
  149.88371096812503],
 'word_piano.wav': [94.89385762555233],
 'word_drums.wav': [93.0421135379224],
 'word_train.wav': [94.89232364424873],
 'word_gunshot.wav': [117.5393414170707, 87.04389766743536, 85.9896281339677],
 'drums_f.wav': [175.40128679262676],
 'churchbells_f.wav': [209.86827953677152, 142.01132911807085],
 'phone_f.wav': [339.61389002701253, 181.50350226893235],
 'gunshot_f.wav': [193.9893552840633, 158.7073756628261],
 'word_doorbell.wav': [95.03023233305127],
 'coin_f.wav': [169.07735629265736],
 'word_paper.wav': [403.8653272096104,
  125.55478248717124,
  150.88784450452

In [94]:
test_file = '0_set_up_and_raw_data/data/stim/low_pass_400/word_crow.wav'
pitch_values = measurePitch(test_file)
syllable_pitches = list(getPitchOfEachSyllable(pitch_values))
pitch_values

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        , 124.1066919 , 123.03159437, 121.9678503 ,
       119.58985297, 117.49307326, 115.23191584, 112.30500126,
       108.4395023 , 104.53543535, 100.95549219,  96.90742777,
        94.06769299,  92.27758581,  90.52565919,  89.46602143,
        88.04409614,  86.96361178,  85.81283524,  85.17778005,
        84.55607414,  84.00185866,  83.77776638,  84.58951916,
        82.31446112,  80.4423577 ,  81.09382947,  87.35469468])

In [75]:
# Get pitches across utterance

# Separate pitches for each syllable

# Compute mean pitch for each syllable

# Store in a dict, key = filename, value = list of pitch values

In [89]:
count = 0
if count:
    print('a')